使用 `torch.autograd`自动微分
===============================================

在训练神经网络时，最常用的算法是 **反向传播**。在此算法中，参数（模型权重）会根据损失函数相对于给定参数的 **梯度** 进行调整。

为了计算这些梯度，PyTorch 有一个内置的微分引擎，称为 torch.autograd。它支持任何计算图的梯度自动计算。

考虑最简单的单层神经网络，输入为 x，参数为 w 和 b，以及一些损失函数。它可以在 PyTorch 中以以下方式定义


In [39]:
import torch

x = torch.ones(5)  # 输入张量 ，训练集
y = torch.zeros(3)  # 输出张量，标签
w = torch.randn(5, 3, requires_grad=True)  # 参数
b = torch.randn(3, requires_grad=True)# 参数
z = torch.matmul(x, w)+b # 公式
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

张量、函数和计算图
==========================================

此代码定义了以下 **计算图**

![](https://pytorch.org/tutorials/_static/img/basics/comp-graph.png)

在此网络中，w 和 b 是 **参数**，我们需要对其进行优化。因此，我们需要能够计算损失函数相对于这些变量的梯度。为了做到这一点，我们设置了这些张量的 requires_grad 属性。


<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>注意</strong></div>

<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">

<p>您可以在创建张量时设置requires_grad的值，或者之后使用x.requires_grad_(True)方法。</code> method.</p>

</div>



我们应用于张量以构建计算图的函数实际上是Function类的对象。此对象知道如何在前向方向计算函数，以及如何在反向传播步骤计算其导数。反向传播函数的引用存储在张量的grad_fn属性中。您可以在文档中找到有关Function的更多信息。[in the
documentation](https://pytorch.org/docs/stable/autograd.html#function).


在 PyTorch 中，grad_fn 是一个 属性，它可以帮助你追踪张量的计算过程。它指示了该张量是如何通过某些操作生成的。具体来说，grad_fn 是 PyTorch 自动微分（autograd）机制的一部分，用于构建计算图，以便计算梯度。

grad_fn 的作用
grad_fn 是一个指向 计算图中操作节点 的引用。每个张量都有一个 grad_fn 属性，指示它是如何通过一系列操作得到的。通过查看 grad_fn，你可以了解张量的来源。
对于 叶子节点（即最初的输入张量），它的 grad_fn 是 None，因为它们没有通过任何操作计算出来。
对于 非叶子节点（即中间计算结果），它们的 grad_fn 会指向生成它们的操作。

In [40]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x0000016D38D4B6D0>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x0000016D38D4B310>


计算梯度
===================

为了优化神经网络中参数的权重，我们需要计算损失函数相对于参数的导数，即，我们需要在x和y的某些固定值下计算 $\frac{\partial loss}{\partial w}$ and
$\frac{\partial loss}{\partial b}$ 。 要计算这些导数，我们调用loss.backward()，然后从w.grad和b.grad中检索值。


In [41]:
loss.backward() # 因为在前面定义了w和b需要计算梯度，所以这一步之后，可以输出w和b的梯度
print(w.grad)
print(b.grad)

tensor([[0.3159, 0.1518, 0.2951],
        [0.3159, 0.1518, 0.2951],
        [0.3159, 0.1518, 0.2951],
        [0.3159, 0.1518, 0.2951],
        [0.3159, 0.1518, 0.2951]])
tensor([0.3159, 0.1518, 0.2951])


<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>注意</strong></div>

<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">

<ul>
<li>我们只能为计算图的叶子节点获取grad属性，这些节点的requires_grad属性设置为True。对于图中的所有其他节点，梯度将不可用。

出于性能原因，我们只能对给定图上的backward执行一次梯度计算。如果我们需要对同一图执行多次backward调用，则需要将retain_graph=True传递给backward调用。</code> </li>
</ul>
```

</div>



禁用梯度跟踪
===========================

默认情况下，所有具有requires_grad=True的张量都跟踪其计算历史并支持梯度计算。但是，在某些情况下我们不需要这样做，例如，当我们训练完模型并只想将其应用于某些输入数据时，即我们只想对网络进行前向计算。我们可以通过用torch.no_grad()块包围我们的计算代码来停止跟踪计算。


如果按照此类讲，一个完整的训练过程，在前向传播时，记录梯度，反向传播时，更新梯度
到预测的时候，如同训练一样，前向传播，但是不需要反向传播，
所以训练过程，可以禁用梯度跟踪

是的，这句话基本是正确的，下面我来详细解释一下其中的关键概念。

1. 前向传播与反向传播：
前向传播（Forward Pass）：这一步计算模型的输出结果。对于神经网络来说，输入数据通过各层网络传递，直到最后的输出层。前向传播会在计算图中记录每一个操作（例如矩阵乘法、激活函数等），并生成每一层的输出。
反向传播（Backward Pass）：这一步通过计算损失函数相对于每个参数的梯度，来更新网络的权重。反向传播是通过计算图进行的，其中的每个操作都包含了计算梯度的必要信息。
2. 梯度计算与禁用梯度跟踪：
梯度计算（Gradient Calculation）：在训练过程中，每当我们进行前向传播并计算损失时，PyTorch 会通过 自动微分（autograd） 自动记录计算图，以便后续计算梯度。在反向传播时，计算图会用来计算梯度并更新网络参数。

禁用梯度跟踪（Disabling Gradient Tracking）：在推理（预测）阶段，我们并不需要更新模型的参数，因此不需要进行反向传播。为了提高效率，在推理阶段，我们可以禁用梯度计算。这可以通过以下方式实现：

使用 torch.no_grad() 上下文管理器。
或者直接将模型的参数设置为 requires_grad=False。
禁用梯度计算的好处是 减少内存开销 和 提高计算速度，因为不需要保存计算图，PyTorch 就不需要为每个张量记录梯度了。

3. 训练与推理的区别：
训练过程：
在训练过程中，前向传播会记录计算图，以便反向传播计算梯度。
反向传播时，计算梯度并更新模型的参数。
推理过程：
在推理过程中，我们仅需要进行前向传播，得到预测结果。
因为不进行反向传播，也不更新模型参数，所以不需要计算梯度。此时禁用梯度跟踪，可以减少内存使用和提高推理效率。
4. 禁用梯度跟踪的实现：
在推理过程中，可以使用 torch.no_grad() 来禁用梯度计算：

In [42]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


另一种获得相同结果的方法是在张量上使用detach()方法。


In [43]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


有一些原因可能导致您想要禁用梯度跟踪。

:   -   将神经网络中的某些参数标记为**冻结参数**.
    -   当您只进行前向传递时，**加快计算速度**，因为对不跟踪梯度的张量的计算会更有效。.


更多关于计算图
============================

从概念上讲，autograd 会在由Function对象组成的有向无环图 (DAG) 中记录数据（张量）和所有执行的操作（以及生成的新的张量）。在此 DAG 中，叶子是输入张量，根是输出张量。通过从根到叶跟踪此图，您可以使用链式规则自动计算梯度。

在前向传递中，autograd 同时执行两件事:

-   运行请求的操作以计算结果张量
-   在 DAG 中维护操作的梯度函数.

当在 DAG 根上调用.backward()时，反向传递开始。然后autograd。:

-   从每个.grad_fn计算梯度,
-   将它们累积在各个张量的.grad属性中
-   使用链式规则，一直传播到叶子张量.

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>注意</strong></div>

<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">

<p>PyTorch 中的 DAG 是动态的需要注意的重要一点是，图是从头开始重新创建的；在每次.backward()调用之后，autograd 都会开始填充一个新的图。这正是允许您在模型中使用控制流语句的原因；如果需要，您可以在每次迭代中更改形状、大小和操作。</p>

</div>



可选阅读：张量梯度和雅可比积
========================================================

在许多情况下，我们有一个标量损失函数，我们需要计算相对于某些参数的梯度。但是，在某些情况下，输出函数是任意张量。在这种情况下，PyTorch 允许您计算所谓的雅可比积，而不是实际的梯度。

对于向量函数 $\vec{y}=f(\vec{x})$, 其中
$\vec{x}=\langle x_1,\dots,x_n\rangle$ 和
$\vec{y}=\langle y_1,\dots,y_m\rangle$,  $\vec{y}$ 相对
 $\vec{x}$ 的梯度由 **雅可比矩阵**给出:

$$\begin{aligned}
J=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)
\end{aligned}$$

PyTorch 允许您为给定的输入向量$v=(v_1 \dots v_m)$计算 **雅可比积** $v^T\cdot J$ 而不是计算雅可比矩阵本身，这是通过使用
$v$作为参数调用backward来实现的.  $v$ 的大小应与原始张量的大小相同，我们希望计算相对于该张量的乘积。


In [44]:
inp = torch.eye(4, 5, requires_grad=True)
inp

tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.]], requires_grad=True)

In [45]:
out = (inp+1).pow(2).t()
out

tensor([[4., 1., 1., 1.],
        [1., 4., 1., 1.],
        [1., 1., 4., 1.],
        [1., 1., 1., 4.],
        [1., 1., 1., 1.]], grad_fn=<TBackward0>)

In [46]:
inp = torch.eye(4, 5, requires_grad=True)
out = (inp+1).pow(2).t()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"First call\n{inp.grad}")
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nSecond call\n{inp.grad}")
inp.grad.zero_()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nCall after zeroing gradients\n{inp.grad}")

First call
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])

Second call
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.]])

Call after zeroing gradients
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])


请注意，当我们第二次使用相同的参数调用backward时，梯度的值不同。发生这种情况是因为在进行backward传播时，PyTorch累积梯度，即计算出的梯度的值会添加到计算图的所有叶子节点的grad属性中。如果您想计算正确的梯度，则需要在之前将grad属性清零。在现实生活中的训练中，优化器可以帮助我们做到这一点。


<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>

<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">

<p>之前我们是在没有参数的情况下调用backward()函数。这实际上等同于调用backward(torch.tensor(1.0))，这是一种在标量值函数（例如神经网络训练期间的损失）的情况下计算梯度的有用方法。.</p>

</div>



------------------------------------------------------------------------
